In [1]:
import cv2 as cv
import numpy as np

from skimage.segmentation import slic
from skimage.color import label2rgb


cap = cv.VideoCapture("video/0.mp4")

frame_space = 5
frame_array = []
for i in range(frame_space):
    ret, first_frame = cap.read()
    mask = np.zeros_like(first_frame)
    mask_pred = np.zeros_like(first_frame)
    prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)
    frame_array.append(prev_gray)

mask[..., 1] = 255
out = cv.VideoWriter('res.mp4', cv.VideoWriter_fourcc('m','p','4','v'), cap.get(cv.CAP_PROP_FPS),
                    (frame_array[0].shape[1], frame_array[0].shape[0]))

while(cap.isOpened()):

    # ret = a boolean return value from getting
    # the frame, frame = the current frame being
    # projected in the video
    ret, frame = cap.read()

    if ret == True:
        # Converts each frame to grayscale - we previously
        # only converted the first frame to grayscale
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        # Calculates dense optical flow by Farneback method
        flow = cv.calcOpticalFlowFarneback(frame_array[0], gray, None,
        0.5, 3, 15, 3, 5, 1.2, 0)

        # Computes the magnitude and angle of the 2D vectors
        magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])

        # Sets image hue according to the optical flow
        # direction
        mask[..., 0] = angle * 180 / np.pi / 2

        # Sets image value according to the optical flow
        # magnitude (normalized)
        mask[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)

        if (mask[..., 2].sum() == 0):
            mask = mask_pred

        # Converts HSV to RGB (BGR) color representation
        rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR)

        frame_array.pop(0)
        frame_array.append(gray)

        # rgb_seg = slic(rgb, n_segments=300, compactness=5)
        # rgb = label2rgb(rgb_seg, rgb, kind = 'avg')

        mask_pred = np.copy(mask)

        tmp = np.ones(rgb.shape)
        rgb = np.where(rgb.sum(axis=2) == 0, 0, 255)
        rgb = rgb[..., np.newaxis]
        rgb = (rgb * tmp).astype(np.uint8)
        out.write(rgb)
    else:
        break

cap.release()
out.release()
cv.destroyAllWindows()